# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [11]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000002  | 3.762376     | 6847177.773672     | 380824.455891 |

| 10        | 11       | 0.000002  | 4.056405     | 6842123.232651     | 364204.576180 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [19]:
model_all.coefficients['value']
model_all.evaluate(validation)['rmse']

353774.2197500632

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [10]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [42]:
best_rss=0.0
best_model=0.0
for l1_penalt in np.logspace(1,7,num=13):
    model=graphlab.linear_regression.create(training,target='price',features=all_features,l2_penalty=0,l1_penalty=l1_penalt,
                                            validation_set=None,verbose=False)
    
    rss=model.evaluate(validation)['rmse']
    if best_rss==0.0:
        best_rss=rss
        best_model=l1_penalt
        
    if rss<best_rss:
        best_model=l1_penalt
        best_rss=rss
        
        
print best_rss,best_model

        

254847.407999 10.0 10.0 254847.407999
254847.408044 31.6227766017 10.0 254847.407999
254847.408186 100.0 10.0 254847.407999
254847.408634 316.227766017 10.0 254847.407999
254847.41005 1000.0 10.0 254847.407999
254847.414528 3162.27766017 10.0 254847.407999
254847.428692 10000.0 10.0 254847.407999
254847.473488 31622.7766017 10.0 254847.407999
254847.615221 100000.0 10.0 254847.407999
254848.064191 316227.766017 10.0 254847.407999
254849.491634 1000000.0 10.0 254847.407999
254854.082379 3162277.66017 10.0 254847.407999
254871.319748 10000000.0 10.0 254847.407999
254847.407999 10.0


In [38]:
model=graphlab.linear_regression.create(training,target='price',features=all_features,l2_penalty=0,l1_penalty=10.0,
                                            validation_set=None,verbose=False)
model.evaluate(validation)['rmse']

254847.41452849974

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [46]:
model=graphlab.linear_regression.create(training,target='price',features=all_features,l2_penalty=0,l1_penalty=10.0,
                                            validation_set=None,verbose=False)
price_predicted = model.predict(testing)
RSS = ((price_predicted-testing['price'])**2).sum()
print RSS

1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [47]:
model.coefficients.print_rows(num_rows=18)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [61]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [48]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [56]:
list_nonz=[]
dict_nonz={}
for l1_penalty in np.logspace(8,10,num=20):
    model=graphlab.linear_regression.create(training,target='price',features=all_features,l2_penalty=0,l1_penalty=l1_penalty
                                           ,validation_set=None,verbose=False)
    list_nonz.append(model.coefficients['value'].nnz())
    dict_nonz[l1_penalty]=model.coefficients['value'].nnz()
    





In [57]:
list_nonz
dict_nonz

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.11147901: 18,
 263665089.87303555: 17,
 335981828.62837881: 17,
 428133239.8719396: 17,
 545559478.11685145: 17,
 695192796.17755914: 17,
 885866790.41008317: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313128: 10,
 3792690190.7322536: 6,
 4832930238.5717525: 5,
 6158482110.6602545: 3,
 7847599703.5146227: 1,
 10000000000.0: 1}

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [66]:
for penalty,nonz in dict_nonz.items():
    if max_nonzeros < nonz:
        continue
        
    l1_penalty_min=penalty

for penalty,nonz in dict_nonz.items():
    if max_nonzeros > nonz:
        l1_penalty_max=penalty  
        
          
l1_penalty_max=3792690190.7322536
l1_penalty_min=2976351441.63

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [77]:
dict_rss={}
dict_nonz={}
for penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model=graphlab.linear_regression.create(training,target='price',features=all_features,l2_penalty=0,
                                           l1_penalty=penalty,validation_set=None,verbose=False)
    predicted_price=model.predict(validation)
    RSS=((predicted_price-validation['price'])**2).sum()
    dict_rss[penalty]=RSS
    dict_nonz[penalty]=model.coefficients['value'].nnz()


dict_rss   

{2976351441.6300001: 966925692362084.5,
 3019316638.9511714: 974019450084556.1,
 3062281836.2723427: 981188367942452.8,
 3105247033.593514: 989328342459474.0,
 3148212230.9146852: 998783211265891.2,
 3191177428.2358561: 1008477167020094.0,
 3234142625.5570273: 1018298780553819.8,
 3277107822.8781986: 1028247992205977.2,
 3320073020.1993699: 1034616909232828.1,
 3363038217.5205412: 1038554735941040.8,
 3406003414.8417125: 1043237237871703.0,
 3448968612.1628838: 1046937488751711.1,
 3491933809.484055: 1051147625612860.9,
 3534899006.8052263: 1055992735342999.1,
 3577864204.1263971: 1060799531763287.8,
 3620829401.4475684: 1065707689498230.1,
 3663794598.7687397: 1069464335425586.5,
 3706759796.089911: 1073504549585599.6,
 3749724993.4110823: 1077632775581416.0,
 3792690190.7322536: 1081867592324110.6}

In [78]:
min(dict_rss, key=dict_rss.get)
dict_nonz

{2976351441.6300001: 10,
 3019316638.9511714: 10,
 3062281836.2723427: 10,
 3105247033.593514: 10,
 3148212230.9146852: 10,
 3191177428.2358561: 10,
 3234142625.5570273: 10,
 3277107822.8781986: 10,
 3320073020.1993699: 8,
 3363038217.5205412: 8,
 3406003414.8417125: 8,
 3448968612.1628838: 7,
 3491933809.484055: 7,
 3534899006.8052263: 7,
 3577864204.1263971: 7,
 3620829401.4475684: 6,
 3663794598.7687397: 6,
 3706759796.089911: 6,
 3749724993.4110823: 6,
 3792690190.7322536: 6}

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?